# [A Gentle Introduction to Serialization for Python](https://machinelearningmastery.com/a-gentle-introduction-to-serialization-for-python/)

Serialization refers to the process of converting a data object (e.g., Python
objects, Tensorflow models) into a format that allows us to store or transmit
the data and then recreate the object when needed using the reverse process of
deserialization.

There are different formats for the serialization of data, such as JSON, XML,
HDF5, and Python’s pickle, for different purposes. JSON, for instance, returns a
human-readable string form, while Python’s pickle library can return a byte
array.

In this post, you will discover how to use two common serialization libraries in
Python to serialize data objects (namely pickle and HDF5) such as dictionaries
and Tensorflow models in Python for storage and transmission.

After completing this tutorial, you will know:

*    Serialization libraries in Python such as pickle and h5py
*    Serializing objects such as dictionaries and Tensorflow models in Python
*    How to use serialization for memoization to reduce function calls

## Overview

The tutorial is divided into four parts; they are:

*   What is serialization, and why do we serialize?
*    Using Python’s pickle library
*    Using HDF5 in Python
*    Comparison between different serialization methods

## What Is Serialization, and Why Should We Care?

Think about storing an integer; how would you store that in a file or transmit
it? That’s easy! We can just write the integer to a file and store or transmit
that file.

But now, what if we think about storing a Python object (e.g., a Python
dictionary or a Pandas DataFrame), which has a complex structure and many
attributes (e.g., the columns and index of the DataFrame, and the data type of
each column)? How would you store it as a file or transmit it to another
computer?

This is where serialization comes in!

Serialization is the process of converting the object into a format that can be
stored or transmitted. After transmitting or storing the serialized data, we are
able to reconstruct the object later and obtain the exact same structure/object,
which makes it really convenient for us to continue using the stored object
later on instead of reconstructing the object from scratch.

In Python, there are many different formats for serialization available. One
common example of hash maps (Python dictionaries) that works across many
languages is the JSON file format which is human-readable and allows us to store
the dictionary and recreate it with the same structure. But JSON can only store
basic structures such as a list and dictionary, and it can only keep strings and
numbers. We cannot ask JSON to remember the data type (e.g., numpy float32 vs.
float64). It also cannot distinguish between Python tuples and lists.

More powerful serialization formats exist. In the following, we will explore two common serialization libraries in Python, namely pickle and h5py.

## Using Python’s Pickle Library

The `pickle` module is part of the Python standard library and implements methods
to serialize (pickling) and deserialize (unpickling) Python objects.

To get started with `pickle`, import it in Python:

In [1]:
import pickle

Afterward, to serialize a Python object such as a dictionary and store the byte
stream as a file, we can use pickle’s `dump()` method.

In [2]:
test_dict = {"Hello": "World!"}
with open("test.pickle", "wb") as outfile:
 	# "wb" argument opens the file in binary mode
	pickle.dump(test_dict, outfile)

The byte stream representing `test_dict` is now stored in the file
`“test.pickle”`!

To recover the original object, we read the serialized byte stream from the file
using pickle’s `load()` method.

In [3]:
with open("test.pickle", "rb") as infile:
 	test_dict_reconstructed = pickle.load(infile)

**Warning**: Only unpickle data from sources you trust, as it is possible for
arbitrary malicious code to be executed during the unpickling process.

Putting them together, the following code helps you to verify that pickle can
recover the same object:

In [4]:
import pickle
 
# A test object
test_dict = {"Hello": "World!"}
 
# Serialization
with open("test.pickle", "wb") as outfile:
    pickle.dump(test_dict, outfile)
print("Written object", test_dict)
 
# Deserialization
with open("test.pickle", "rb") as infile:
    test_dict_reconstructed = pickle.load(infile)
print("Reconstructed object", test_dict_reconstructed)
 
if test_dict == test_dict_reconstructed:
    print("Reconstruction success")

Written object {'Hello': 'World!'}
Reconstructed object {'Hello': 'World!'}
Reconstruction success


Besides writing the serialized object into a pickle file, we can also obtain the
object serialized as a bytes-array type in Python using pickle’s `dumps()`
function:

In [5]:
test_dict_ba = pickle.dumps(test_dict)      # b'\x80\x04\x95\x15…

Similarly, we can use pickle’s load method to convert from a bytes-array type
back to the original object:

In [6]:
test_dict_reconstructed_ba = pickle.loads(test_dict_ba)

One useful thing about pickle is that it can serialize almost any Python object,
including user-defined ones, such as the following:

In [7]:
import pickle
 
class NewClass:
    def __init__(self, data):
        print(data)
        self.data = data
 
# Create an object of NewClass
new_class = NewClass(1)
 
# Serialize and deserialize
pickled_data = pickle.dumps(new_class)
reconstructed = pickle.loads(pickled_data)
 
# Verify
print("Data from reconstructed object:", reconstructed.data)

1
Data from reconstructed object: 1


Pickle can even serialize Python functions since functions are first-class objects in Python:

In [8]:
import pickle
 
def test():
    return "Hello world!"
 
# Serialize and deserialize
pickled_function = pickle.dumps(test)
reconstructed_function = pickle.loads(pickled_function)
 
# Verify
print (reconstructed_function()) #prints “Hello, world!”

Hello world!


Therefore, we can make use of pickle to save our work. For example, a trained
model from Keras or scikit-learn can be serialized by pickle and loaded later
instead of re-training the model every time we use it. The following shows you
how we can build a LeNet5 model to recognize the **MNIST handwritten digits**
using Keras, then serialize the trained model using pickle. Afterward, we can
reconstruct the model without training it again, and it should produce exactly
the same result as the original model:

In [9]:
import pickle
 
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, AveragePooling2D, Dropout, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
 
# Load MNIST digits
(X_train, y_train), (X_test, y_test) = mnist.load_data()
 
# Reshape data to (n_samples, height, wiedth, n_channel)
X_train = np.expand_dims(X_train, axis=3).astype("float32")
X_test = np.expand_dims(X_test, axis=3).astype("float32")
 
# One-hot encode the output
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
 
# LeNet5 model
model = Sequential([
    Conv2D(6, (5,5), input_shape=(28,28,1), padding="same", activation="tanh"),
    AveragePooling2D((2,2), strides=2),
    Conv2D(16, (5,5), activation="tanh"),
    AveragePooling2D((2,2), strides=2),
    Conv2D(120, (5,5), activation="tanh"),
    Flatten(),
    Dense(84, activation="tanh"),
    Dense(10, activation="softmax")
])
 
# Train the model
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
earlystopping = EarlyStopping(monitor="val_loss", patience=4, restore_best_weights=True)
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32, callbacks=[earlystopping])
 
# Evaluate the model
print(model.evaluate(X_test, y_test, verbose=0))
 
# Pickle to serialize and deserialize
pickled_model = pickle.dumps(model)
reconstructed = pickle.loads(pickled_model)
 
# Evaluate again
print(reconstructed.evaluate(X_test, y_test, verbose=0))



2022-07-20 12:03:03.619798: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-20 12:03:06.815570: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/targets/x86_64-linux/lib/
2022-07-20 12:03:06.815659: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/targets/x86_64-linux/lib/
2022-07-20 12:03:06.815713: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuf

Epoch 1/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.1577 - accuracy: 0.9538 - val_loss: 0.0748 - val_accuracy: 0.9757
Epoch 2/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0666 - accuracy: 0.9795 - val_loss: 0.0481 - val_accuracy: 0.9848
Epoch 3/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0518 - accuracy: 0.9842 - val_loss: 0.0476 - val_accuracy: 0.9847
Epoch 4/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0387 - accuracy: 0.9880 - val_loss: 0.0406 - val_accuracy: 0.9870
Epoch 5/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0341 - accuracy: 0.9890 - val_loss: 0.0474 - val_accuracy: 0.9856
Epoch 6/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0284 - accuracy: 0.9912 - val_loss: 0.0406 - val_accuracy: 0.9875
Epoch 7/100
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0253 - accuracy: 0.9919 - val_loss: 0.0406 -

Note that the evaluation scores from the original and reconstructed models are
tied out perfectly in the last two lines.

While pickle is a powerful library, it still does have its own limitations to
what can be pickled. For example, live connections such as database connections
and opened file handles cannot be pickled. This issue arises because
reconstructing these objects requires pickle to re-establish the connection with
the database/file, which is something pickle cannot do for you (because it needs
appropriate credentials and is out of the scope of what pickle is intended for).

## Using HDF5 in Python

Hierarchical Data Format 5 (HDF5) is a binary data format. The h5py package is a
Python library that provides an interface to the HDF5 format. From h5py docs,
HDF5 “lets you store huge amounts of numerical data, and easily manipulate that
data from Numpy.”

What HDF5 can do better than other serialization formats is store data in a file
system-like hierarchy. You can store multiple objects or datasets in HDF5, like
saving multiple files in the file system. You can also read a particular dataset
from HDF5, like reading one file from the file system without concerning the
other. If you’re using pickle for this, you will need to read and write
everything each time you load or create the pickle file. Hence HDF5 is
advantageous for huge amounts of data that can’t fit entirely into memory.

To get started with h5py, you first need to install the h5py library, which you
can do using:

In [ ]:
%pip install h5py

Note: you may need to restart the kernel to use updated packages.


We can then get started with creating our first dataset!

In [ ]:
import h5py
 
with h5py.File("test.hdf5", "w") as file:
    dataset = file.create_dataset("test_dataset", (100,), dtype="i4")

This creates a new dataset in the file `test.hdf5` named `“test_dataset,”` with a
shape of (100, ) and a type int32. `h5py` datasets follow a Numpy syntax so that
you can do slicing, retrieval, get shape, etc., similar to Numpy arrays.

To retrieve a specific index:

In [ ]:
dataset[0]  #retrieves element at index 0 of dataset

ValueError: Invalid dataset identifier (invalid dataset identifier)

To get a slice from index 0 to index 10 of a dataset:

In [ ]:
dataset[:10]

If you initialized the h5py file object outside of a with statement, remember to
close the file as well!  

To read from a previously created HDF5 file, you can
open the file in “r” for read mode or “r+” for read/write mode:

In [ ]:
with h5py.File("test.hdf5", "r") as file:
    print (file.keys()) #gets names of datasets that are in the file
    dataset = file["test_dataset"]

To organize your HDF5 file, you can use groups:

In [ ]:
with h5py.File("test.hdf5", "w") as file:
    # creates new group_1 in file
    file.create_group("group_1")
    group1 = file["group_1"]
    # creates dataset inside group1
    group1.create_dataset("dataset1", shape=(10,))
    # to access the dataset
    dataset = file["group_1"]["dataset1"]

Another way to create groups and files is by specifying the path to the dataset
you want to create, and h5py will create the groups on that path as well (if
they don’t exist):

In [ ]:
with h5py.File("test.hdf5", "w") as file:
    # creates dataset inside group1
    file.create_dataset("group1/dataset1", shape=(10,))

The two snippets of code both create group1 if it has not been created previously and then a dataset1 within group1.

## HDF5 in Tensorflow

To save a model in Tensorflow Keras using HDF5 format, we can use the `save()` function of the model with a filename having extension .h5, like the following:

In [ ]:
from tensorflow import keras
 
# Create model
model = keras.models.Sequential([
 	keras.layers.Input(shape=(10,)),
 	keras.layers.Dense(1)
])
 
model.compile(optimizer="adam", loss="mse")
 
# using the .h5 extension in the file name specifies that the model
# should be saved in HDF5 format
model.save("my_model.h5")

To load the stored HDF5 model, we can also use the function from Keras directly:

In [ ]:
...
model = keras.models.load_model("my_model.h5")
 
# to check that the model has been successfully reconstructed
print(model.summary)

One reason we don’t want to use pickle for a Keras model is that we need a more
flexible format that does not tie to a particular version of Keras. If we
upgraded our Tensorflow version, the model object might change, and pickle may
fail to give us a working model. Another reason is to keep only the essential
data for our model. For example, if we check the HDF5 file my_model.h5 created
in the above, we see these are stored:

In [ ]:
import h5py
 
with h5py.File("my_model.h5", "r") as infile:
    print(infile["/model_weights/dense/dense/kernel:0"][:])

And in HDF5, the metadata is stored alongside the data. Keras stored the
network’s architecture in a JSON format in the metadata. Hence we can reproduce
our network architecture as follows:

In [ ]:
import json
import h5py
 
with h5py.File("my_model.h5", "r") as infile:
    for key in infile.attrs.keys():
        formatted = infile.attrs[key]
        if key.endswith("_config"):
            formatted = json.dumps(json.loads(formatted), indent=4)
        print(f"{key}: {formatted}")

The model config (i.e., the architecture of our neural network) and training
config (i.e., the parameters we passed into the compile() function) are stored
as a JSON string. In the code above, we use the json module to reformat it to
make it easier to read. It is recommended to save your model as HDF5 rather than
just your Python code because, as we can see above, it contains more detailed
information than the code on how the network was constructed.  

## Comparing Between Different Serialization Methods

In the above, we saw how pickle and h5py can help serialize our Python data.

We can use pickle to serialize almost any Python object, including user-defined ones and functions. But pickle is not language agnostic. You cannot unpickle it outside Python. There are even 6 versions of pickle developed so far, and older Python may not be able to consume the newer version of pickle data.

On the contrary, HDF5 is cross-platform and works well with other language such as Java and C++. In Python, the h5py library implemented the Numpy interface to make it easier to manipulate the data. The data can be accessed in a different language because the HDF5 format supports only the Numpy data types such as float and strings. We cannot store arbitrary objects such as a Python function into HDF5.

## Further Reading

This section provides more resources on the topic if you are looking to go deeper.
### Articles

*    Serialization from C# programming guide, https://docs.microsoft.com/en-us/dotnet/csharp/programming-guide/concepts/serialization/
*    Save and load Keras models, https://www.tensorflow.org/guide/keras/save_and_serialize

### Libraries

*    pickle, https://docs.python.org/3/library/pickle.html
*    h5py, https://docs.h5py.org/en/stable/

### APIs

*    Tensorflow tf.keras.layers.serialize, https://www.tensorflow.org/api_docs/python/tf/keras/layers/serialize
*    Tensorflow tf.keras.models.load_model, https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model
*    Tensorflow tf.keras.models.save_model, https://www.tensorflow.org/api_docs/python/tf/keras/models/save_model

### Summary

In this post, you discovered what serialization is and how to use libraries in Python to serialize Python objects such as dictionaries and Tensorflow Keras models. You have also learned the advantages and disadvantages of two Python libraries for serialization (pickle, h5py).

Specifically, you learned:

*    what is serialization, and why it is useful
*    how to get started with pickle and h5py serialization libraries in Python
*    pros and cons of different serialization methods
